In [ ]:
import pandas as pd
df=pd.read_csv('/content/dialogueText_file1.csv')

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 243105 entries, 0 to 243104
Data columns (total 2 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   dialogueID  243105 non-null  object
 1   text        243105 non-null  object
dtypes: object(2)
memory usage: 3.7+ MB


In [ ]:

data = df.sample(frac=0.05)


In [ ]:
# Display basic information about the dataset
print(data.info())
print(data.head())

<class 'pandas.core.frame.DataFrame'>
Index: 12155 entries, 171795 to 156120
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   dialogueID  12155 non-null  object
 1   text        12155 non-null  object
dtypes: object(2)
memory usage: 284.9+ KB
None
        dialogueID                                     text
171795  144798.tsv  httpubuntuforumsorgarchiveindexphpthtml
102514  334943.tsv          the invers rename kgd into sgdm
219286  241033.tsv        ask the question dont ask the ask
26846   106870.tsv                                   hellou
109289    5321.tsv                                   agreed


In [ ]:
# Optional: Clean the dataset (e.g., remove missing values, duplicates, etc.)
data = data.dropna().drop_duplicates()

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 12154 entries, 171795 to 156120
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   dialogueID  12154 non-null  object
 1   text        12154 non-null  object
dtypes: object(2)
memory usage: 284.9+ KB


In [ ]:
from sklearn.model_selection import train_test_split

# Split the data into training and validation sets
train_data, val_data = train_test_split(data, test_size=0.2, random_state=42)

# Save the splits for future use
train_data.to_csv('train_data.csv', index=False)
val_data.to_csv('val_data.csv', index=False)


In [ ]:
from transformers import BertTokenizer, BertModel
import torch

# Load the tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [ ]:

# Function to embed text
def embed_text(text):
    inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True, max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1).squeeze().numpy()

In [ ]:
# Apply the embedding function to the data
data['embeddings'] = data['text'].apply(embed_text)

In [ ]:
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 82.1 MB/s eta 0:00:00


In [ ]:
import numpy as np
import faiss
# Convert embeddings to a numpy array
embedding_matrix = np.vstack(data['embeddings'].values)

# Build the FAISS index
index = faiss.IndexFlatL2(embedding_matrix.shape[1])
index.add(embedding_matrix)

In [ ]:
# Save the index and the corresponding dialogue IDs
faiss.write_index(index, "faiss_index.bin")
data[['dialogueID', 'text']].to_csv("dialogue_texts.csv", index=False)

In [ ]:
# Save the model
model.save_pretrained('bert_embedding_model')

# Save the tokenizer
tokenizer.save_pretrained('bert_embedding_model')


('bert_embedding_model/tokenizer_config.json',
 'bert_embedding_model/special_tokens_map.json',
 'bert_embedding_model/vocab.txt',
 'bert_embedding_model/added_tokens.json')